In [37]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/seattle/calendar.csv
/kaggle/input/seattle/reviews.csv
/kaggle/input/seattle/listings.csv


In [38]:
listings_df = pd.read_csv('/kaggle/input/seattle/listings.csv')
calendar_df = pd.read_csv('/kaggle/input/seattle/calendar.csv')

In [39]:
print(listings_df.shape)
print(listings_df.info())
print(listings_df.head())
print(calendar_df.shape)
print(calendar_df.info())
print(calendar_df.head())

(3818, 92)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
id                                  3818 non-null int64
listing_url                         3818 non-null object
scrape_id                           3818 non-null int64
last_scraped                        3818 non-null object
name                                3818 non-null object
summary                             3641 non-null object
space                               3249 non-null object
description                         3818 non-null object
experiences_offered                 3818 non-null object
neighborhood_overview               2786 non-null object
notes                               2212 non-null object
transit                             2884 non-null object
thumbnail_url                       3498 non-null object
medium_url                          3498 non-null object
picture_url                         3818 non-null object
xl_picture_url               

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1393570 entries, 0 to 1393569
Data columns (total 4 columns):
listing_id    1393570 non-null int64
date          1393570 non-null object
available     1393570 non-null object
price         934542 non-null object
dtypes: int64(1), object(3)
memory usage: 42.5+ MB
None
   listing_id        date available   price
0      241032  2016-01-04         t  $85.00
1      241032  2016-01-05         t  $85.00
2      241032  2016-01-06         f     NaN
3      241032  2016-01-07         f     NaN
4      241032  2016-01-08         f     NaN


In [40]:
listings_df_new = listings_df[['id','host_id', 'price', 'latitude', 'longitude', 'city', 'neighbourhood'
                                                      , 'host_since', 'host_is_superhost'
                                                      , 'room_type', 'bed_type', 'beds', 'bedrooms', 'bathrooms'
                                                      , 'minimum_nights', 'cancellation_policy', 'instant_bookable']]

In [41]:
listing_calendar_df = listings_df_new.merge(calendar_df, left_on = 'id', right_on = 'listing_id', suffixes = ['_listing', '_calendar'])

In [42]:
listing_calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393570 entries, 0 to 1393569
Data columns (total 21 columns):
id                     1393570 non-null int64
host_id                1393570 non-null int64
price_listing          1393570 non-null object
latitude               1393570 non-null float64
longitude              1393570 non-null float64
city                   1393570 non-null object
neighbourhood          1241730 non-null object
host_since             1392840 non-null object
host_is_superhost      1392840 non-null object
room_type              1393570 non-null object
bed_type               1393570 non-null object
beds                   1393205 non-null float64
bedrooms               1391380 non-null float64
bathrooms              1387730 non-null float64
minimum_nights         1393570 non-null int64
cancellation_policy    1393570 non-null object
instant_bookable       1393570 non-null object
listing_id             1393570 non-null int64
date                   1393570 non-null 

In [43]:
listing_calendar_df = listing_calendar_df.drop(['id', 'listing_id'], axis = 1)
listing_calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393570 entries, 0 to 1393569
Data columns (total 19 columns):
host_id                1393570 non-null int64
price_listing          1393570 non-null object
latitude               1393570 non-null float64
longitude              1393570 non-null float64
city                   1393570 non-null object
neighbourhood          1241730 non-null object
host_since             1392840 non-null object
host_is_superhost      1392840 non-null object
room_type              1393570 non-null object
bed_type               1393570 non-null object
beds                   1393205 non-null float64
bedrooms               1391380 non-null float64
bathrooms              1387730 non-null float64
minimum_nights         1393570 non-null int64
cancellation_policy    1393570 non-null object
instant_bookable       1393570 non-null object
date                   1393570 non-null object
available              1393570 non-null object
price_calendar         934542 non-null

In [44]:
listing_calendar_df.isnull().sum()

host_id                     0
price_listing               0
latitude                    0
longitude                   0
city                        0
neighbourhood          151840
host_since                730
host_is_superhost         730
room_type                   0
bed_type                    0
beds                      365
bedrooms                 2190
bathrooms                5840
minimum_nights              0
cancellation_policy         0
instant_bookable            0
date                        0
available                   0
price_calendar         459028
dtype: int64

A few columns contains missing values. Instead of drop all missing values, which will cause troubles for further analysis, I will impute the missing values. There are many common methods for missing value imputation. However, after review the dataset, I think the common methods are not the best options for this datasest. For example, host_is_superhost column contains 730 missing values. The superhost is a feature that AirBnB gave to the host who provided excellent services and had been approved by the visitors every 3 months. This is a flag that provide by the AirBnB. If a host doesn't have the value of is the superhost that highly represents this host is not superhost. Meanwhile, the records that don't have superhost data, don't have the host_since data either. I assume these records are from new hosts. Therefore, using the 'f' value to fillin all the missing values in host_is_superhost column instead of using common missing value imputation is more properity. This logic will apply to other missing value imputation. Using the values that meet the situation of the real business rather than the common mean, median or mode imputation.

Fill Missing Values

In [45]:
listing_calendar_df.loc[(listing_calendar_df['city'] == '西雅图') |(listing_calendar_df['city'] == 'seattle')|(listing_calendar_df['city'] == 'Seattle '), 'city'] = 'Seattle'

In [46]:
listing_calendar_df.loc[listing_calendar_df.neighbourhood.isnull(), 'neighbourhood'] = listing_calendar_df.loc[listing_calendar_df.neighbourhood.isnull(), 'city']

In [47]:
listing_calendar_df.loc[listing_calendar_df.host_is_superhost.isnull(), 'host_is_superhost'] = 'f'

In [48]:
listing_calendar_df.loc[listing_calendar_df.host_since.isnull(), ].groupby('host_id').date.agg('min')
listing_calendar_df.loc[listing_calendar_df.host_since.isnull(), 'host_since'] = '2016-01-04'

In [49]:
listing_calendar_df.loc[listing_calendar_df.price_calendar.isnull() > 0, "price_calendar"] = listing_calendar_df.loc[listing_calendar_df.price_calendar.isnull() > 0, "price_listing"] 

In [50]:
listing_calendar_df.loc[listing_calendar_df.price_calendar.isnull() > 0, "price_calendar"].isnull()

Series([], Name: price_calendar, dtype: bool)

In [51]:
listing_calendar_df.drop(['price_listing'], axis = 1)
listing_calendar_df = listing_calendar_df.rename(columns = {'price_calendar':'price'})
listing_calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393570 entries, 0 to 1393569
Data columns (total 19 columns):
host_id                1393570 non-null int64
price_listing          1393570 non-null object
latitude               1393570 non-null float64
longitude              1393570 non-null float64
city                   1393570 non-null object
neighbourhood          1393570 non-null object
host_since             1393570 non-null object
host_is_superhost      1393570 non-null object
room_type              1393570 non-null object
bed_type               1393570 non-null object
beds                   1393205 non-null float64
bedrooms               1391380 non-null float64
bathrooms              1387730 non-null float64
minimum_nights         1393570 non-null int64
cancellation_policy    1393570 non-null object
instant_bookable       1393570 non-null object
date                   1393570 non-null object
available              1393570 non-null object
price                  1393570 non-nul

In [52]:
listing_calendar_df.loc[listing_calendar_df.bedrooms.isnull(), 'bedrooms'] = 0

In [53]:
listing_calendar_df.loc[listing_calendar_df.beds.isnull(), 'bed_type'].unique()
listing_calendar_df.loc[listing_calendar_df.beds.isnull(), 'beds'] = 1

In [54]:
listing_calendar_df.loc[listing_calendar_df.bathrooms.isnull(), 'bathrooms'] = 0

In [55]:
listing_calendar_df.isnull().sum()

host_id                0
price_listing          0
latitude               0
longitude              0
city                   0
neighbourhood          0
host_since             0
host_is_superhost      0
room_type              0
bed_type               0
beds                   0
bedrooms               0
bathrooms              0
minimum_nights         0
cancellation_policy    0
instant_bookable       0
date                   0
available              0
price                  0
dtype: int64

In [56]:
listing_calendar_df = listing_calendar_df.drop(['host_id', 'price_listing'], axis = 1)

Convert data type

In [58]:
listing_calendar_df.host_since.dtypes

dtype('O')

In [59]:
def data_conversion(df):
    '''
    INPUT: df - pandas dataframe you want to convert data type
    OUTPUT: df - a new dataframe that has the following characteristics:
    1. all the dates are datetime data type
    2. all the binary variables are bool type
    3. all the price variables are numeric
    '''
    for col in df.columns:
        if (df[col].dtype == 'O'):
            if (df[col].str.match("[1-2][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9]").sum() > 0):
                df[col] = pd.to_datetime(df[col], format='%Y-%m-%d', errors='ignore')
            elif (list(df[col].unique()) == ['t', 'f'] or list(df[col].unique()) == ['f', 't']):
                bl_convert = {'t': True, 'f': False}
                df[col] = df[col].replace(bl_convert)
            elif (df[col].str.contains("\$", na=False).sum() > 0):
                df[col] = pd.to_numeric(df[col].str.replace('\$|\,', ''))
        else:
            continue
    return df

In [60]:
listing_calendar_df = data_conversion(listing_calendar_df)
listing_calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1393570 entries, 0 to 1393569
Data columns (total 17 columns):
latitude               1393570 non-null float64
longitude              1393570 non-null float64
city                   1393570 non-null object
neighbourhood          1393570 non-null object
host_since             1393570 non-null datetime64[ns]
host_is_superhost      1393570 non-null bool
room_type              1393570 non-null object
bed_type               1393570 non-null object
beds                   1393570 non-null float64
bedrooms               1393570 non-null float64
bathrooms              1393570 non-null float64
minimum_nights         1393570 non-null int64
cancellation_policy    1393570 non-null object
instant_bookable       1393570 non-null bool
date                   1393570 non-null datetime64[ns]
available              1393570 non-null bool
price                  1393570 non-null float64
dtypes: bool(3), datetime64[ns](2), float64(6), int64(1), object(5)
memor

In [96]:
listing_calendar_avail_df = listing_calendar_df[['date', 'available']]
listing_calendar_avail_df = listing_calendar_avail_df.set_index(['date'])
listing_calendar_avail_g_df = listing_calendar_avail_df.resample('D').sum()
listing_calendar_avail_g_df.index
sns.lineplot(x = listing_calendar_avail_g_df.index, y = "available", data = listing_calendar_avail_g_df)

NameError: name 'index' is not defined

In [63]:
listing_calendar_df_lm = listing_calendar_df[['price', 'available', 'date', 'city', 'neighbourhood', 'host_since', 'host_is_superhost', 'room_type',
                                             'bed_type', 'beds', 'bedrooms', 'bathrooms'
                                            , 'minimum_nights', 'cancellation_policy', 'instant_bookable']]

In [64]:
listing_calendar_df_lm['host_since_days'] = (listing_calendar_df_lm['date'] - listing_calendar_df_lm['host_since']).astype('timedelta64[D]')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
#need to convert the date into category
listing_calendar_df_lm['month'] = pd.DatetimeIndex(listing_calendar_df_lm['date']).month.astype('category')

In [66]:
listing_calendar_df_lm['month'].dtype

CategoricalDtype(categories=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ordered=False)

In [67]:
listing_calendar_df_lm = pd.get_dummies(listing_calendar_df_lm[['price', 'available', 'month', 'city', 'neighbourhood', 'host_since_days', 'host_is_superhost', 'room_type',
                                             'bed_type', 'beds', 'bedrooms', 'bathrooms'
                                            , 'minimum_nights', 'cancellation_policy', 'instant_bookable']], prefix_sep='_')

In [68]:
listing_calendar_df_lm.head()

,price,available,host_since_days,host_is_superhost,beds,bedrooms,bathrooms,minimum_nights,instant_bookable,month_1,...,room_type_Private room,room_type_Shared room,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict
0,85.0,True,1607.0,False,1.0,1.0,1.0,1,False,1,...,0,0,0,0,0,0,1,0,1,0
1,85.0,True,1608.0,False,1.0,1.0,1.0,1,False,1,...,0,0,0,0,0,0,1,0,1,0
2,85.0,False,1609.0,False,1.0,1.0,1.0,1,False,1,...,0,0,0,0,0,0,1,0,1,0
3,85.0,False,1610.0,False,1.0,1.0,1.0,1,False,1,...,0,0,0,0,0,0,1,0,1,0
4,85.0,False,1611.0,False,1.0,1.0,1.0,1,False,1,...,0,0,0,0,0,0,1,0,1,0


In [69]:
def clean_fit_linear_mod(df, response_col, test_size=.3, rand_state=123):
    '''
    INPUT:
    df - a dataframe holding all the variables of interest
    response_col - a string holding the name of the column 
    test_size - a float between [0,1] about what proportion of data should be in the test dataset
    rand_state - an int that is provided as the random state for splitting the data into training and test 
    
    OUTPUT:
    test_score - float - r2 score on the test data
    train_score - float - r2 score on the test data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    '''

    X = df.drop(response_col, axis = 1)
    y = df[response_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = rand_state)
    lm_model = LinearRegression(normalize = True)
    lm_model.fit(X_train, y_train)
    y_train_pred = lm_model.predict(X_train)
    y_test_pred = lm_model.predict(X_test)
    train_score = r2_score(y_train, y_train_pred)
    test_score = r2_score(y_test, y_test_pred)
    return test_score, train_score, lm_model, X_train, X_test, y_train, y_test

In [70]:
test_score, train_score, lm_model, X_train, X_test, y_train, y_test = clean_fit_linear_mod(listing_calendar_df_lm, 'price')

In [71]:
print(test_score, train_score)

0.5921062977120043 0.5922151561526257


In [72]:
lm_model.coef_

array([ 1.81926788e+01,  6.47519981e-04,  1.03847643e+01,  1.31151094e+01,
        4.02235232e+01,  3.27689125e+01, -9.55333871e-03, -4.27701460e+00,
       -1.85716594e+13, -1.85716594e+13, -1.85716594e+13, -1.85716594e+13,
       -1.85716594e+13, -1.85716594e+13, -1.85716594e+13, -1.85716594e+13,
       -1.85716594e+13, -1.85716594e+13, -1.85716594e+13, -1.85716594e+13,
        9.54985106e+13,  9.54985106e+13,  9.54985106e+13,  9.54985106e+13,
        6.45560514e+13,  6.45560514e+13,  6.45560514e+13,  6.45560514e+13,
        6.45560514e+13,  6.45560514e+13,  6.45560514e+13,  6.45560514e+13,
        6.45560514e+13,  6.45560514e+13,  6.45560514e+13,  6.45560514e+13,
        6.45560514e+13,  6.45560514e+13,  6.45560514e+13,  6.45560514e+13,
        6.45560514e+13,  6.45560514e+13,  6.45560514e+13,  6.45560514e+13,
        6.45560514e+13,  6.45560514e+13,  6.45560514e+13,  6.45560514e+13,
        6.45560514e+13,  6.45560514e+13,  6.45560514e+13,  6.45560514e+13,
        6.45560514e+13,  

In [73]:
lm_model.intercept_

-382567262638730.7